# Comment charger les modèles de ML batch 

# 1. Analyse prédictive des ventes

In [3]:
from pyspark.ml.regression import LinearRegressionModel
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

# Créez une session Spark
spark = SparkSession.builder.appName("Load_Models").getOrCreate()

# Charger le modèle
loaded_model = LinearRegressionModel.load("hdfs://hadoop-namenode:9000/mllib_models/sales_prediction_model")

# Créer un DataFrame avec les caractéristiques pour la prédiction
new_data = spark.createDataFrame([(2023, 10, 9)], ["Year", "Month", "Day"])

assembler = VectorAssembler(inputCols=["Year", "Month", "Day"], outputCol="features")

new_data = assembler.transform(new_data)

new_data.show()

# Utiliser le modèle pour faire des prédictions
predictions = loaded_model.transform(new_data)

# Afficher les prédictions
predictions.show()

+----+-----+---+-----------------+
|Year|Month|Day|         features|
+----+-----+---+-----------------+
|2023|   10|  9|[2023.0,10.0,9.0]|
+----+-----+---+-----------------+

+----+-----+---+-----------------+-----------------+
|Year|Month|Day|         features|       prediction|
+----+-----+---+-----------------+-----------------+
|2023|   10|  9|[2023.0,10.0,9.0]|25.45667985623163|
+----+-----+---+-----------------+-----------------+



# 2. Segmentation de la clientèle 

In [6]:
#charger le modèle
from pyspark.ml.clustering import KMeansModel
from pyspark.ml.feature import StandardScaler

loaded_model = KMeansModel.load("hdfs://hadoop-namenode:9000/mllib_models/customers_segmentation_model")

# Chargement des données depuis le fichier CSV
orders_df = spark.read.csv("hdfs://hadoop-namenode:9000/orders.csv", header=True, inferSchema=True)
nouveaux_clients_df = spark.read.csv("hdfs://hadoop-namenode:9000/customers.csv", header=True, inferSchema=True)

# Regroupement des données d'achat par client
customer_purchase_data = orders_df.groupBy("CustomerKey").sum("Quantity").withColumnRenamed("sum(Quantity)", "TotalPurchase")

# Fusion des données de comportement d'achat avec les données clients
nouveaux_clients_df = nouveaux_clients_df.join(customer_purchase_data, on="CustomerKey", how="inner")

# Sélection des caractéristiques pertinentes pour la PCA
feature_columns = ["TotalPurchase"]

# Création d'un vecteur d'assemblage des caractéristiques
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
nouveaux_clients_pretraites = assembler.transform(nouveaux_clients_df)

# Mise à l'échelle des caractéristiques (si nécessaire)
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=True)
scaler_model = scaler.fit(nouveaux_clients_pretraites)
nouveaux_clients_pretraites = scaler_model.transform(nouveaux_clients_pretraites)

# Utilisation du modèle K-Means chargé pour prédire les clusters des nouveaux clients
predictions = loaded_model.transform(nouveaux_clients_pretraites)

# Affichage des résultats
predictions.select("CustomerKey", "TotalPurchase", "prediction").show()


+-----------+-------------+----------+
|CustomerKey|TotalPurchase|prediction|
+-----------+-------------+----------+
|         28|          507|         3|
|         27|          186|         0|
|         26|          712|         3|
|         12|          234|         0|
|         22|          602|         3|
|          1|          377|         0|
|         13|         1155|         2|
|          6|          395|         0|
|         16|         1419|         2|
|          3|          394|         0|
|         20|          644|         3|
|          5|          391|         0|
|         19|          374|         0|
|         15|          246|         0|
|          9|          657|         3|
|         17|          624|         3|
|          4|          504|         3|
|          8|         2301|         1|
|         23|          245|         0|
|          7|          346|         0|
+-----------+-------------+----------+
only showing top 20 rows



# 3. Recommandation de produits

In [8]:
from pyspark.ml.recommendation import ALSModel
from pyspark.sql import functions as F


#Chargement du modèle
loaded_model = ALSModel.load("hdfs://hadoop-namenode:9000/mllib_models/recommendation_system_model")

customers_df = spark.read.csv("hdfs://hadoop-namenode:9000/customers.csv", header=True, inferSchema=True)

user_id = 12
user_recs = loaded_model.recommendForUserSubset(customers_df.where(F.col("CustomerKey") == user_id), 5)
user_recs.show()

+-----------+--------------------+
|CustomerKey|     recommendations|
+-----------+--------------------+
|         12|[{96, 5.726271}, ...|
+-----------+--------------------+



# 4. Optimisation de la chaîne d’approvisionnement
comme c'est un modèle de regression linéaire il faut juste faire comme le premier